# Disclaimer
This material was prepared as an account of work sponsored by an agency of the United States Government.  Neither the United States Government nor the United States Department of Energy, nor Battelle, nor any of their employees, nor any jurisdiction or organization that has cooperated in the development of these materials, makes any warranty, express or implied, or assumes any legal liability or responsibility for the accuracy, completeness, or usefulness or any information, apparatus, product, software, or process disclosed, or represents that its use would not infringe privately owned rights. Reference herein to any specific commercial product, process, or service by trade name, trademark, manufacturer, or otherwise does not necessarily constitute or imply its endorsement, recommendation, or favoring by the United States Government or any agency thereof, or Battelle Memorial Institute. The views and opinions of authors expressed herein do not necessarily state or reflect those of the United States Government or any agency thereof.

PACIFIC NORTHWEST NATIONAL LABORATORY operated by BATTELLE for the UNITED STATES DEPARTMENT OF ENERGY under Contract DE-AC05-76RL01830.

# Step-by-Step Application of NukeLM Classifier to CSV of Documents

Import needed packages and define project directory root

In [ ]:
from pathlib import Path

import torch
import transformers
from datasets import load_dataset

from nukelm.analyze.serve import serve_documents


project_dir = Path.cwd().parent

Parameters for serving the model

In [ ]:
model_path_or_name = str(project_dir / "data" / "06_models" / "Binary Classification")
tokenizer_path_or_name = "roberta-large"

input_path = str(project_dir / "data" / "01_raw" / "OSTI" / "binary_label_test_filtered_trimmed_1000.csv")

use_cuda = True
batch_size = 4

output_path = str(project_dir / "data" / "07_model_output" / "nukelm-output.csv")

Load the model into memory, and optionally onto the GPU

In [ ]:
config = transformers.AutoConfig.from_pretrained(model_path_or_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_path_or_name, config=config)
if tokenizer_path_or_name is None:
    tokenizer_path_or_name = model_path_or_name
tokenizer = transformers.AutoTokenizer.from_pretrained(tokenizer_path_or_name)

try:
    category_labels = [config.id2label[i] for i in range(len(config.id2label))]
except AttributeError:
    category_labels = None  # type: ignore

if use_cuda and torch.cuda.is_available():
    model.cuda()

Load the dataset from CSV

In [ ]:
dataset = load_dataset("csv", data_files=input_path)["train"]  # datasets puts a single data_files in "train"
assert "text" in dataset.column_names

Apply the model to the "text" column of the dataset

In [ ]:
def _serve_documents(examples):
    """Factory for `serve_documents` for use with `dataset.map`."""
    return serve_documents(
        examples["text"],
        model=model,
        tokenizer=tokenizer,
        pool_strategies=["CLS", "MAX", "MEAN"],
        embed=True,
        classify=True,
        category_labels=category_labels,
        use_cuda=use_cuda and torch.cuda.is_available(),
    )


dataset = dataset.map(
    _serve_documents,
    batched=True,
    batch_size=batch_size,
)

Inspect the columns of the output

In [ ]:
dataset

Inspect a single example of the output

In [ ]:
dataset[0]

Save the output to a CSV

In [ ]:
dataset.remove_columns(["CLS", "MAX", "MEAN"]).to_csv(output_path)